## e-Statデータ前処理（5歳階級別小地域人口データ）

In [1]:
# インポート
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import glob
import os
pd.set_option('display.max_columns', 200)

In [2]:
# e-Statデータのフォルダパス
data_path = '../raw_data/男女，年齢（5歳階級）別人口，平均年齢及び総年齢－町丁・字等_2020/'

# csvファイルパス
csv_path = '/*.csv'

# 出力先
output_data_path = '../output/data_preprocessed/e-stat/'

In [38]:
# 前処理
def estat_population_data_preprocessing(data_path, csv_path):
    """
    各都道府県のe-Stat人口データを順番に読み込み、結合などの前処理を行う
    """

    # データ読み込み
    estat_file_names = glob.glob(data_path + csv_path)
    estat_list = []
    for file_name in estat_file_names:
        estat = pd.read_csv(file_name, encoding = 'cp932')
        estat_list.append(estat)

    # 都道府県ごとのデータフレームを結合
    estat_df_pre = pd.concat(estat_list)
    estat_df_pre.reset_index(drop = True, inplace = True)

    # 細かく分かれている住所名を結合し、「住所」列に追加
    estat_df_pre['住所'] = estat_df_pre['都道府県名'] + estat_df_pre['市区町村名'] + estat_df_pre['大字・町名'] + estat_df_pre['字・丁目名']
    # 「字・丁目名」がNullの場合、住所もNullになってしまうため、以下コードで追加
    estat_df_pre.fillna({'住所': estat_df_pre['都道府県名'] + estat_df_pre['市区町村名'] + estat_df_pre['大字・町名']}, inplace = True)
    # 「大字・町名」もNullの場合、住所もNullになってしまうため、以下コードで追加
    estat_df_pre.fillna({'住所': estat_df_pre['都道府県名'] + estat_df_pre['市区町村名']}, inplace = True)

    # 必要な行だけを残す
    drop_cols_estat = ['地域階層レベル', '秘匿処理', '秘匿先情報', '合算地域', '総年齢', '平均年齢']
    estat_df = estat_df_pre.drop(drop_cols_estat, axis = 1)

    # 列の並び替え
    col_addr = estat_df.pop('住所')
    estat_df.insert(loc = 7 , column= '住所', value = col_addr)
    
    # 列名の置換
    def replace_column_name(column_name):
        new_column_name = column_name.replace('歳', '').replace('（再掲）', '').replace('以上', '～')
        return new_column_name
    
    new_columns = [replace_column_name(col) for col in estat_df.columns]
    estat_df = estat_df.rename(columns = dict(zip(estat_df.columns, new_columns)))
    
    # NaNの削除
    start_col_name = '総数'
    end_col_name = '20～69'
    def dropna_in_df(df, start_col_name, end_col_name):
        dropna_cols = df.loc[:, start_col_name:end_col_name].columns
        df_dropna = df.dropna(subset = dropna_cols)
        return df

    estat_df = dropna_in_df(estat_df, start_col_name, end_col_name)
    
    return estat_df

In [39]:
estat_df = estat_population_data_preprocessing(data_path, csv_path)
estat_df

C:\Users\jpnkb\AppData\Local\Temp\ipykernel_12608\2002789543.py:11: DtypeWarning: Columns (40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  estat = pd.read_csv(file_name, encoding = 'cp932')
C:\Users\jpnkb\AppData\Local\Temp\ipykernel_12608\2002789543.py:11: DtypeWarning: Columns (31,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  estat = pd.read_csv(file_name, encoding = 'cp932')


,男女,市区町村コード,町丁字コード,都道府県名,市区町村名,大字・町名,字・丁目名,住所,総数,0～4,5～9,10～14,15～19,20～24,25～29,30～34,35～39,40～44,45～49,50～54,55～59,60～64,65～69,70～74,75～79,80～84,85～89,90～94,95～99,100～,年齢「不詳」,15未満,15～64,65～,75～,85～,20～69
0,総数,26203,3009,京都府,綾部市,口上林,十倉中町,京都府綾部市口上林十倉中町,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,総数,26203,7207,京都府,綾部市,吉美,工業団地,京都府綾部市吉美工業団地,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,総数,26203,8060,京都府,綾部市,綾部,味方町,京都府綾部市綾部味方町,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,総数,26203,8063,京都府,綾部市,綾部,青野町,京都府綾部市綾部青野町,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,男,26203,3009,京都府,綾部市,口上林,十倉中町,京都府綾部市口上林十倉中町,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111910,男,25211,250,滋賀県,湖南市,高松町,NaN,滋賀県湖南市高松町,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111911,女,25211,250,滋賀県,湖南市,高松町,NaN,滋賀県湖南市高松町,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111912,総数,25204,840,滋賀県,近江八幡市,津田町,NaN,滋賀県近江八幡市津田町,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
111913,男,25204,840,滋賀県,近江八幡市,津田町,NaN,滋賀県近江八幡市津田町,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
estat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111915 entries, 0 to 111914
Data columns (total 37 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   男女       111915 non-null  object
 1   市区町村コード  111915 non-null  int64 
 2   町丁字コード   111915 non-null  object
 3   都道府県名    111915 non-null  object
 4   市区町村名    111915 non-null  object
 5   大字・町名    111817 non-null  object
 6   字・丁目名    68837 non-null   object
 7   住所       111915 non-null  object
 8   総数       111915 non-null  int64 
 9   0～4      111915 non-null  object
 10  5～9      111915 non-null  object
 11  10～14    111915 non-null  object
 12  15～19    111915 non-null  object
 13  20～24    111915 non-null  object
 14  25～29    111915 non-null  object
 15  30～34    111915 non-null  object
 16  35～39    111915 non-null  object
 17  40～44    111915 non-null  object
 18  45～49    111915 non-null  object
 19  50～54    111915 non-null  object
 20  55～59    111915 non-null  object
 21  60～64    1

In [47]:
# csv出力
os.makedirs(output_data_path, exist_ok = True)
estat_df.to_csv(output_data_path + 'estat_population_2020_preprocessed.csv', index = False, encoding = 'utf-8_sig')

### csv出力後、excel操作で人口比率を算出